# **Proyecto 3 - Quora Insincere Questions**

AJA, PROFESOR ARAQUE, AGARRESE PORQUE LO QUE VA A VER A CONTINUACIÓN ES ALTO PROYECTO, OK? LO SIENTO MUCHO PERO DESPUÉS DE VERLO 
USTED NO ME VA A SALIR CON COSAS KIKE "No y tal, les faltó un poco mejorar esto y explicar mejor aquello" PORQUE ESTE ES LITERAL 
MI ÚLTIMO PROYECTO DE PROGRAMACIÓN DE LA CARRERA Y ME LO VOY A TRIPEAR COMO NO TIENE IDEA

## 1) Preprocesamiento de los Datos

Basado en el Kaggle de [Dieter](https://www.kaggle.com/christofhenkel) ([How to: Preprocessing when using embeddings](https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings)) se hará un preprocesamiento al conjunto de acuerdo al embedding a utilizar, esto para poder acercar el vocubulario lo más cercano posible al embedding

### 1.1) Importación de librerías

In [ ]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas() # Esto es para poder ver progreso de los datos que se están procesando porque 
              # sino no ando cloro de que está pasando y me pieldo

Cargamos el ganado:

In [ ]:
train = pd.read_csv("../input/quora-insincere-questions-classification/train.csv")
test = pd.read_csv("../input/quora-insincere-questions-classification/test.csv")
print("Train shape : ",train.shape)
print("Test shape : ",test.shape)

## 1.2) Función auxiliar

La siguiente función trifásica es para mantener un seguimiento del vocabulario de entrenamiento, éste va a lo largo de todo el texto contando las ocurrencias las palabras que están contenido dentro de éste.

In [ ]:
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

### 1.3) Primera cita

En la primera cita es cuando uno obtiene de verdad la primera impresión de la persona que lo encarpa a uno, esta es nuestra primera cita con el conjunto de datos, ay kemosión 🧡

In [ ]:
sentences = train["question_text"].progress_apply(lambda x: x.split()).values
vocab = build_vocab(sentences)
print({k: vocab[k] for k in list(vocab)[:5]})

"How did Quebec nationalists see", el hecho de que sean nacionalistas no implican que sean ciegos 🙄

### 1.4) Importando el embedding que tal

Bueno, ahora hay que importar el beta que realmente importa, embedding que se va a usar para el modelo.

In [ ]:
from gensim.models import KeyedVectors

news_path = '../input/quora-insincere-questions-classification/embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
embeddings_index = KeyedVectors.load_word2vec_format(news_path, binary=True)

### 1.5) Hora de saber que tanto B!"@-hit tiene el embedding 

La función a continuación nos va a permitir ver la intersección que hay entre el vocabulario (o nuestros datos) y los embeddings, de manera tal de que se pueda limpiar y ajustar más el vocabulario a nuestras necesidades. 

Se denotará a "oov" como aquellas palabras que están fuera del vocabulario (out of vocabulary)

In [ ]:
import operator 

def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

In [ ]:
oov = check_coverage(vocab,embeddings_index)

Sólamente el 32% del vocabulario tiene embeddings, de manera que aproximadamente 21% de los datos que se tienen no sirven de nada. Esto no puede ser así vale, ¿Cómo en un proyecto tan arrecho como este podemos permitir esto? Nonono que vaina es pues.

A continuación se muestran las primeras 10 palabras dentro de la variable oov:

In [ ]:
oov[:10]

Acá podemos ver como pequeñas palabras que no aportan mucho valor como "to", "a", "of" y "and" hay por coñazo en nuestro conjunto de datos pero está fuera de los embeddings de GoogleNews, esto evidentemente hay que eliminarlo ya que no aporta valor en absoluto para lo que se quiere. A su vez, podemos notar como hay varias palabras que incluyen signo de interrogación en ellas como si fuesen una sola, por lo que es momento de ver si este se encuentra en los embeddings, de manera tal que sea necesario mantenerlo o no.

In [ ]:
'?' in embeddings_index

In [ ]:
'&' in embeddings_index

Acá podemos ver que los embeddings de GoogleNews no contienen el signo de interrogación pero si inluyen el simbolo de ampersand. De hecho, luego de una larga y exhaustiva búsqueda en Mister Google, resulta que sus embeddings [no contienen signos de puntuación](https://www.reddit.com/r/MLQuestions/comments/aaha9g/can_word_embeddings_like_glove_fasttext_and/ecs115z/), entonces dado que sobran, a nadie les importa y Google siempre tiene la razón vamos a quitarlos xd 

*En realidad es para poder mejorar el vocabulario para que tenga más palabras en el conjunto de datos que concuerden con los embeddings*

### 1.6) Se acerca el verano y las manos... A la obra porque hay que trabajar osi osi

In [ ]:
def clean_text(x):

    x = str(x)
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in '&':
        x = x.replace(punct, f' {punct} ')
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct, '')
    return x

La función anterior nos permitirá cumplir lo anterior descrito.

Ahora vamos a ver qlq con qlq:

In [ ]:
train["question_text"] = train["question_text"].progress_apply(lambda x: clean_text(x))
sentences = train["question_text"].apply(lambda x: x.split())
vocab = build_vocab(sentences)

In [ ]:
oov = check_coverage(vocab,embeddings_index)

![](https://i.ibb.co/ckLKc7j/descargar.jpg)VAMOOOOOOSSSSSS, así es que es vale nojombre, somos altos hackers y el que diga lo contrario es un nvidioso (osea un oso que trabaja en Nvidia).

Bueno, aquí lo que pasó es que se duplicó la cantidad de vocabulario que se encuentra en los embeddings arreglando los signos de puntuación. Veamos que tenemos ahora:

In [ ]:
oov[:10]

Bien, ahora sucede que hay varios números en el aire, veamos más detalladamente esto:

In [ ]:
for i in range(10):
    print(embeddings_index.index2entity[i])

Bien, lo que sucede es que los números que son mayores a 9 son transformados en numerales (#), vamos a arreglar eso porque de pana que así no se puede:

In [ ]:
import re

def clean_numbers(x):

    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

In [ ]:
train["question_text"] = train["question_text"].progress_apply(lambda x: clean_numbers(x))
sentences = train["question_text"].progress_apply(lambda x: x.split())
vocab = build_vocab(sentences)

In [ ]:
oov = check_coverage(vocab,embeddings_index)

Bien, no es una mejora tan sustancial como la anterior pero es algo vale, no se puede pedir tantos cambios tan radicales en esta vida, poco a poco, a paso de fracasadores... khé

![](https://images7.memedroid.com/images/UPLOADED830/5ddbb05163918.jpeg)

Vemos que nos quedó en las palabras que están fuera del vocabulario:

In [ ]:
oov[:20]

Bien, hay un problema y es que aun quedan palabras pequeñas como "to" que no están en los embeddings de GoogleNews y son un grupo generoso que evidentemente no está aportando nada en absoluto. Ya nos encargaremos de eso.

Mientras tanto la siguiente función multifásica radioactiva nos permite hacer varias cosas, como son quitar palabras mal escritas, acronimos, palabras de inglés británico, así como nombres de redes sociales que centraremos en "social medium":

In [ ]:
def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


mispell_dict = {'colour':'color',
                'centre':'center',
                'didnt':'did not',
                'doesnt':'does not',
                'isnt':'is not',
                'shouldnt':'should not',
                'favourite':'favorite',
                'travelling':'traveling',
                'counselling':'counseling',
                'theatre':'theater',
                'cancelled':'canceled',
                'labour':'labor',
                'organisation':'organization',
                'wwii':'world war 2',
                'citicise':'criticize',
                'instagram': 'social medium',
                'whatsapp': 'social medium',
                'snapchat': 'social medium'

                }
mispellings, mispellings_re = _get_mispell(mispell_dict)

def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)

Una vez hecho esto, vamos a quitar las palabras 'a','to','of' y 'and' de nuestro conjunto de datos porque no están en los embeddings de GoogleNews, entonces no es que estén aportando mucho ps

In [ ]:
train["question_text"] = train["question_text"].progress_apply(lambda x: replace_typical_misspell(x))
sentences = train["question_text"].progress_apply(lambda x: x.split())
to_remove = ['a','to','of','and']
sentences = [[word for word in sentence if not word in to_remove] for sentence in tqdm(sentences)]
vocab = build_vocab(sentences)

In [ ]:
oov = check_coverage(vocab,embeddings_index)

![YEAAAAAAAAAH](https://i.ibb.co/Z1ydGW5/descargar-1.jpg)

**LISTOOOOOOO, conseguimos que cerca del 99% del texto del conjunto de datos aparezca en los embeddings de GoogleNews**

Y si vemos la variable oov:

In [ ]:
oov[:20]

No es que haya soluciones así como que bien rapidito para ellas pero bueno, se hace lo que se puede y pa' lante ps. 

Hablando claro 99% es bien decente

## 2) Preparando todo para que empiece la fiesta

![](https://cdn.dopl3r.com//media/memes_files/quedan-pocos-dias-para-salvar-el-semestre-a-trabajar-kdkdkd-mematicnet-vw5RH.jpg)

*Si profe, estamos haciendo este proyecto a última hora, por favor no nos juzgue.*

Una vez habiendo procesado los datos para que se apreveche al máximo el uso de los embeddings, se procederá a hacer el setup de lo que será nuestra red neuronal recurrente para el analisis de las preguntas de Quora

### 2.1) Importando las librerías que nos van a hacer todo juas juas

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from sklearn.model_selection import train_test_split

### 2.2) Obteniendo un conjunto de validación

Sacaremos un pequeño conjunto de validación de nuestro conjunto de entrenamiento:

In [ ]:
train_df, val_df = train_test_split(train, test_size=0.1)

Ahora cargaremos bien en un diccionario los embeddings de GoogleNews:

In [ ]:
word2vecDict = embeddings_index
embeddings_index = dict()
for word in word2vecDict.wv.vocab:
    embeddings_index[word] = word2vecDict.word_vec(word)
print('Loaded %s word vectors.' % len(embeddings_index))

Una vez cargado el diccionario con los vectores de las palabras, se convertiran cada una de las oraciones a un vector con los valores correspondientes del embedding de GoogleNews, al final lo que se tendrá, será una matriz que contenga todos los valores de las oraciones.

### 2.3) Funciones auxiliares

In [ ]:
# Convert values to embeddings
def text_to_array(text):
    empyt_emb = np.zeros(300)
    text = text[:-1].split()[:30]
    embeds = [embeddings_index.get(x, empyt_emb) for x in text]
    embeds+= [empyt_emb] * (30 - len(embeds))
    return np.array(embeds)

# train_vects = [text_to_array(X_text) for X_text in tqdm(train_df["question_text"])]
val_vects = np.array([text_to_array(X_text) for X_text in tqdm(val_df["question_text"][:3000])])
val_y = np.array(val_df["target"][:3000])


La siguiente función, es una mausqueherramienta misteriosa que nos ayudará más tarde, esta permitirá generar batch de tamaño de 128 para el entrenamiento de la red

In [ ]:
# Data providers
batch_size = 128

def batch_gen(train_df):
    n_batches = math.ceil(len(train_df) / batch_size)
    while True: 
        train_df = train_df.sample(frac=1.)  # Shuffle the data.
        for i in range(n_batches):
            texts = train_df.iloc[i*batch_size:(i+1)*batch_size, 1]
            text_arr = np.array([text_to_array(text) for text in texts])
            yield text_arr, np.array(train_df["target"][i*batch_size:(i+1)*batch_size])

## 3) Definiendo el modelo que tal



### 3.1) Importación de librerías

In [ ]:
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Dense, Bidirectional
import tensorflow as tf

### 3.2) Creación del modelo que tal

In [ ]:
#inp = tf.keras.layers.Input(shape=(10,))
#emb = tf.keras.layers.Embedding(20, 4)(inp)
#x2 = tf.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNGRU(64, return_sequences=True))(emb)
#max_pl = tf.keras.layers.GlobalMaxPooling1D()(x2)
#x = tf.keras.layers.Dense(16, activation="relu")(max_pl)
#x = tf.keras.layers.Dropout(0.1)(x)
#output = tf.keras.layers.Dense(1, activation="sigmoid")(x)

#model = tf.keras.models.Model(inputs=inp, outputs=output)

model = tf.keras.models.Sequential()
x = tf.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(64, return_sequences=True),input_shape=(30, 300))
model.add(x)
y = tf.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(64))
model.add(y)
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

In [ ]:

mg = batch_gen(train_df)



model.fit_generator(mg, epochs=20,
                    steps_per_epoch=1000,
                    validation_data=(val_vects, val_y),
                    verbose=True)

Limpiaremos la RAM

In [ ]:
import gc
del val_vects, val_y, train_df, val_df, oov, sentences, to_remove, vocab, mispell_dict, news_path
gc.collect()

In [ ]:

# prediction part
batch_size = 256
def batch_gens(test_df):
    n_batches = math.ceil(len(test_df) / batch_size)
    for i in range(n_batches):
        texts = test_df.iloc[i*batch_size:(i+1)*batch_size, 1]
        text_arr = np.array([text_to_array(text) for text in texts])
        yield text_arr


all_preds = []
for x in tqdm(batch_gens(test)):
    all_preds.extend(model.predict(x).flatten())
    gc.collect()

In [ ]:
y_te = (np.array(all_preds) > 0.5).astype(np.int)

submit_df = pd.DataFrame({"qid": test["qid"], "prediction": y_te})
submit_df.to_csv("submission.csv", index=False)

import matplotlib.pyplot as plt
submit_df['prediction'].plot(kind='hist', bins=100)
plt.xlabel('prediction')